In [1]:
# Import required packages
import time
import numpy as np
import pandas as pd
from tabulate import tabulate
import torch
from tqdm import trange
from pafik.solver import Solver
from pafik.settings import (
    DEFAULT_NSF,
    DEFULT_SOLVER,
)

TEST_PAFIK = True
TEST_IKFLOW = True
NUM_POSES = 3000  # 100
NUM_SOLS = 400  # 1000
BATCH_SIZE = 5000
SUCCESS_THRESHOLD = (5e-3, 2)
STD = 0.1
USE_NSF_ONLY = False
METHOD_OF_SELECT_REFERENCE_POSTURE = "knn"
WORKDIR = "."

In [2]:
solver_param = DEFAULT_NSF if USE_NSF_ONLY else DEFULT_SOLVER
solver_param.workdir = WORKDIR
solver_param.select_reference_posture_method = METHOD_OF_SELECT_REFERENCE_POSTURE
solver = Solver(solver_param=solver_param)

(
    avg_l2,
    avg_ang,
    avg_inference_time,
    success_rate,
) = solver.evaluate_ikp_iterative(
    NUM_POSES,
    NUM_SOLS,
    std=STD,
    batch_size=BATCH_SIZE,
    success_threshold=SUCCESS_THRESHOLD,
)  # type: ignore
print(
    tabulate(
        [
            [
                avg_l2,
                np.rad2deg(avg_ang),
                avg_inference_time,
                success_rate,
            ]
        ],
        headers=[
            "avg_l2",
            "avg_ang",
            "avg_inference_time",
            f"success_rate ({METHOD_OF_SELECT_REFERENCE_POSTURE})",
        ],
    )
)


WorldModel::LoadRobot: /home/luca/.cache/jrl/temp_urdfs/panda_arm_hand_formatted_link_filepaths_absolute.urdf
joint mimic: no multiplier, using default value of 1 
joint mimic: no offset, using default value of 0 
URDFParser: Link size: 17
URDFParser: Joint size: 12
LoadAssimp: Loaded model /home/luca/miniconda3/envs/test/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link0.dae (59388 verts, 20478 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/envs/test/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link1.dae (37309 verts, 12516 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/envs/test/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link2.dae (37892 verts, 12716 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/envs/test/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link3.dae (42512 verts, 14233 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/envs/test/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link4.dae (435

100%|██████████| 240/240 [02:09<00:00,  1.86it/s]


                 l2           ang
count  1.200000e+06  1.200000e+06
mean   2.851631e-03  1.364338e+00
std    6.706650e-03  4.078244e+00
min    5.162123e-06  5.124690e-02
25%    1.199415e-03  4.171484e-01
50%    1.884752e-03  6.473975e-01
75%    2.861600e-03  1.092205e+00
max    1.044579e+00  1.799797e+02
    avg_l2    avg_ang    avg_inference_time    success_rate (knn)
----------  ---------  --------------------  --------------------
0.00285163    1.36434                 0.048                 0.853


In [ ]:
import numpy as np
import pandas as pd
from tqdm import trange
from sklearn.cluster import AgglomerativeClustering

def cluster_based_on_distance(a, dist_thresh=1):
    kmeans= AgglomerativeClustering(n_clusters=None, distance_threshold=dist_thresh).fit(a)
    return a[np.sort(np.unique(kmeans.labels_, return_index=True)[1])]

In [ ]:
NUM_POSES = 10000
N_NEIGHBORS = 500
NUM_SOLS = N_NEIGHBORS
LAMBDA = (0.005, 0.05)
STD = 0.25

num_poses = NUM_POSES
num_sols = NUM_SOLS
n_neighbors = N_NEIGHBORS
verbose = True
batch_size = 5000
lambda_ = LAMBDA
joint_cofig_distance_thres_rads = 2

In [ ]:
# P = (NUM_POSES, m)
_, P = solver._robot.sample_joint_angles_and_poses(
    n=num_poses, return_torch=False
)

# F = (NUM_POSES * N_NEIGHBORS, r)
F = solver._F[
                solver.nearest_neighnbor_P.kneighbors(
                    np.atleast_2d(P), n_neighbors=n_neighbors, return_distance=False
                ).flatten()
            ]

P_expand_dim = np.repeat(np.expand_dims(P, axis=1), n_neighbors, axis=1).reshape(-1, P.shape[-1])
F.shape, P_expand_dim.shape

In [ ]:
solver.base_std = STD
J_hat = solver.solve_batch(P_expand_dim, F, 1, batch_size=batch_size, verbose=verbose)
l2, ang = solver.evaluate_pose_error(J_hat, P_expand_dim, return_all=True)

J_hat = J_hat.reshape(NUM_POSES, N_NEIGHBORS, -1)
l2 = l2.reshape(NUM_POSES, N_NEIGHBORS)
ang = ang.reshape(NUM_POSES, N_NEIGHBORS)

n_clusters = np.empty((NUM_POSES))
for i in trange(NUM_POSES):
    # print("Filter by l2 and ang")
    J_candidate = J_hat[i][(l2[i] < lambda_[0]) & (ang[i] < lambda_[1])]
    # print(J_candidate.shape)
    # print("J_candidate.shape", J_candidate.shape)
    # print(f"Cluster by joint distance")
    if J_candidate.shape[0] < 2:
        n_clusters[i] = 0    
    else:
        J_filtered = cluster_based_on_distance(J_candidate, dist_thresh=joint_cofig_distance_thres_rads)
        # print("J_filtered.shape", J_filtered.shape)
        n_clusters[i] = J_filtered.shape[0]
    
df = pd.DataFrame({
    'n_clusters': n_clusters
})
df.describe()

In [ ]:
import torch
from ikflow.utils import set_seed
from ikflow.model_loading import get_ik_solver
from jkinpylib.evaluation import solution_pose_errors

set_seed()
# Build IKFlowSolver and set weights
ik_solver, _ = get_ik_solver("panda__full__lp191_5.25m")
l2 = np.zeros((NUM_SOLS, len(P)))
ang = np.zeros((NUM_SOLS, len(P)))
J_flow = torch.empty((NUM_SOLS, len(P), 7), dtype=torch.float32, device="cpu")

if num_poses < num_sols:
    for i in trange(num_poses):
        J_flow[:, i, :] = ik_solver.solve(
            P[i],
            n=num_sols,
            latent_scale=STD,
            refine_solutions=False,
            return_detailed=False,
        ).cpu()  # type: ignore

        l2[:, i], ang[:, i] = solution_pose_errors(
            ik_solver.robot, J_flow[:, i, :], P[i]
        )
else:
    for i in trange(num_sols):
        J_flow[i] = ik_solver.solve_n_poses(
            P, latent_scale=STD, refine_solutions=False, return_detailed=False
        ).cpu()
        l2[i], ang[i] = solution_pose_errors(ik_solver.robot, J_flow[i], P)
    
# J_flow = (NUM_SOLS, NUM_POSES, 7)
n_clusters = np.empty((NUM_POSES))
for i in trange(NUM_POSES):
    J_row = J_flow[:, i, :]
    # print("Filter by l2 and ang")
    J_candidate = J_row[(l2[:, i] < lambda_[0]) & (ang[:, i] < lambda_[1])]
    # print("J_candidate.shape", J_candidate.shape)
    # print(f"Cluster by joint distance")
    if J_candidate.shape[0] < 2:
        n_clusters[i] = 0    
    else:
        J_filtered = cluster_based_on_distance(J_candidate, dist_thresh=joint_cofig_distance_thres_rads)
        # print("J_filtered.shape", J_filtered.shape)
        n_clusters[i] = J_filtered.shape[0]

df = pd.DataFrame({
    'n_clusters': n_clusters
})
df.describe()